# Validation Tools


In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns

%matplotlib inline

## Get Datas

In [2]:
hr_df = pd.read_csv('./HR_comma_sep.csv')

hr_df.head(3)

FileNotFoundError: [Errno 2] No such file or directory: './HR_comma_sep.csv'

## EDA

In [ ]:
# Missing Values
hr_df.isna().sum()

In [ ]:
sns.heatmap(hr_df.isnull(), cmap="ocean") # cmap : white, gray, etc...

In [ ]:
# How many employees are in DB
hr_df.shape

#### Q. How many employees left company ?

In [ ]:
hr_df.left.unique()

In [ ]:
left = hr_df[hr_df.left == 1]

print(f'{left.shape[0]} left the company')

In [ ]:
retained = hr_df[hr_df.left == 0]

print(f'{retained.shape[0]} remained in the company')

How much the others parameters influence the target [ left]?

In [ ]:
grp_left = hr_df.groupby('left').mean()
grp_left

In [ ]:
grp_left['satisfaction_level'].plot(kind='bar')

Brief Conclusions:
+ *satisfaction_level* relatively low for leaving employees(44%) **VS** (66%) for retained empls.
+ *average_montly_hours* higher for leaving employees
+ *Promotion Last 5 Years* higher rated for retained employees

### Frequencies Tables 
using Crosstab

In [ ]:
sal_left_df = pd.crosstab(hr_df.salary, hr_df.left)

sal_left_df.columns = ['left', 'stayed']
sal_left_df

In [ ]:
sal_left_df.plot(kind='bar')

In [ ]:
pd.crosstab(hr_df.Department, hr_df.left).plot(kind='bar')

Correlation

In [ ]:
hr_df.corr()

In [ ]:
sns.heatmap(hr_df.corr())

I will choose to keep:


0. Satisfaction level
1. Average monthly hours
2. Promotion last 5 y
3. Salary
4. Departement

In [ ]:
hr_df.columns

In [ ]:
df = hr_df[['satisfaction_level', 
            'average_montly_hours', 
            'promotion_last_5years', 
            'Department', 
            'salary',
            'left'
            ]]
df.head()

### Preprocessing of datas

*Categorical Variables*

In [ ]:
df.columns[df.dtypes == "object"]

In [ ]:
categoricals = df.columns[df.dtypes == "object"]

In [ ]:
pd.get_dummies(df, categoricals)

In [ ]:
pd.get_dummies(df, categoricals, drop_first= True)

In [ ]:
df.salary.unique()

In [ ]:
df.salary = df.salary.replace(
    {
    'low': 1, 
    'medium': 2, 
    'high': 3
    }
)

In [ ]:
df.salary.unique()

In [ ]:
pd.get_dummies(df, columns=['Department'], drop_first=True)

In [ ]:
df_with_dummies = pd.get_dummies(df, columns=['Department'], drop_first=True)

### Features and target

X & y

In [ ]:
X = df_with_dummies.drop(columns='left', axis=1)
y = df_with_dummies.left


### Validation Tools

#### 1. Train Test Split

In [ ]:
# 1. Train Test Split

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test =  train_test_split(X, y, test_size=0.2, random_state=50)

Log Reg

In [ ]:
# Log Reg
from sklearn.linear_model import LogisticRegression

log = LogisticRegression().fit(X_train, y_train)
log.score(X_test, y_test)

KNeighbors Classifier

In [3]:
# KNeighbors Classifier

from sklearn.neighbors import KNeighborsClassifier

KNN = KNeighborsClassifier().fit(X_train, y_train)
KNN.score(X_test, y_test)

NameError: name 'X_train' is not defined

Decision Tree

In [4]:
# Decision Tree
from sklearn.tree import DecisionTreeClassifier

Tree = DecisionTreeClassifier().fit(X_train, y_train)

Tree.score(X_test, y_test)

NameError: name 'X_train' is not defined

#### 2. Cross Validation

In [5]:
from sklearn.model_selection import cross_val_score, validation_curve, learning_curve


# 
def my_cross_val(model, param, range, cv):

    val_score = []

    for k in range:
        score = cross_val_score(
            model(param=k), 
            X_train, 
            y_train, 
            cv=cv, 
            scoring = 'accuracy'
            ).mean()
        
        val_score.append(score)
        
    return val_score

Log Reg Cross Val

In [6]:

# Log Reg
val_score = []

for k in range(1, 100):
    score = cross_val_score(
        LogisticRegression(max_iter=k), 
        X_train, 
        y_train, 
        cv=5, 
        scoring = 'accuracy'
        ).mean()
    
    val_score.append(score)
    
    plt.plot(val_score)

NameError: name 'LogisticRegression' is not defined

Cross Val Knn

In [7]:


# KNN
val_score = []

for k in range(1, 100):
    score = cross_val_score(
        KNeighborsClassifier(n_neighbors=k),
        X_train,
        y_train,
        cv=5,
        scoring='accuracy'
    ).mean()

    val_score.append(score)

    plt.plot(val_score)

NameError: name 'X_train' is not defined

Cross Val Dec Tree

In [8]:

# Decision Tree
val_score = []

for k in range(1, 100):
    score = cross_val_score(
        DecisionTreeClassifier(max_depth=k),
        X_train,
        y_train,
        cv=5,
        scoring='accuracy'
    ).mean()

    val_score.append(score)

    plt.plot(val_score)

NameError: name 'X_train' is not defined

#### 3. Validation Curve

In [9]:
log_ = LogisticRegression()
k = np.arange(1, 100)


train_score, val_score = validation_curve(
    log_,
    X_train,
    y_train,
    param_name="max_iter",
    param_range=k,
    cv=5,
    scoring='accuracy'
)

plt.plot(k, train_score.mean(axis=1), label="train score")
plt.plot(k, val_score.mean(axis=1), label="val score")

plt.xlabel('max_iter')
plt.ylabel('score')
plt.legend()

NameError: name 'LogisticRegression' is not defined

Validation Curve KNN

In [10]:
KNN_ = KNeighborsClassifier()
k = np.arange(1, 100)


train_score, val_score = validation_curve(
    KNN_,
    X_train,
    y_train,
    param_name="n_neighbors",
    param_range=k,
    cv=5,
    scoring='accuracy'
)

plt.plot(k, train_score.mean(axis=1), label="train score")
plt.plot(k, val_score.mean(axis=1), label="val score")

plt.xlabel('max_iter')
plt.ylabel('score')
plt.legend()

NameError: name 'X_train' is not defined

Validation curve Decision Tree

In [11]:
DTree_ = DecisionTreeClassifier()
k = np.arange(1, 100)


train_score, val_score = validation_curve(
    DTree_,
    X_train,
    y_train,
    param_name="max_depth",
    param_range=k,
    cv=5,
    scoring='accuracy'
)

plt.plot(k, train_score.mean(axis=1), label="train score")
plt.plot(k, val_score.mean(axis=1), label="val score")

plt.xlabel('max_depth')
plt.ylabel('score')
plt.legend()

NameError: name 'X_train' is not defined

#### 4. Learning Curve

Log Reg

In [12]:
N, train_score, val_score = learning_curve(
                            log,
                            X_train,
                            y_train,
                            train_sizes=np.linspace(.2, 1.0, num=5),
                            cv = 5
)

plt.plot(N, train_score.mean(axis=1), label="Train score")
plt.plot(N, val_score.mean(axis=1), label="Val score")


plt.xlabel("train_size")
plt.ylabel("score")

plt.legend()

NameError: name 'log' is not defined

Learning Curve KNN

In [13]:
N, train_score, val_score = learning_curve(
    KNN,
    X_train,
    y_train,
    train_sizes=np.linspace(.2, 1.0, num=5),
    cv=5
)

plt.plot(N, train_score.mean(axis=1), label="Train score")
plt.plot(N, val_score.mean(axis=1), label="Val score")


plt.xlabel("train_size")
plt.ylabel("score")

plt.legend()

NameError: name 'KNN' is not defined

Learning Curve DTree

In [14]:
N, train_score, val_score = learning_curve(
    Tree,
    X_train,
    y_train,
    train_sizes=np.linspace(.2, 1.0, num=5),
    cv=5
)

plt.plot(N, train_score.mean(axis=1), label="Train score")
plt.plot(N, val_score.mean(axis=1), label="Val score")


plt.xlabel("train_size")
plt.ylabel("score")

plt.legend()

NameError: name 'Tree' is not defined

#### 5 Confusion Matrix

In [15]:
from sklearn.metrics import confusion_matrix, classification_report

real_labels_of_test_dataset = y_test
predicted_labels = log.predict(X_test)
display(confusion_matrix(
    real_labels_of_test_dataset,
    predicted_labels
    ))

cm = confusion_matrix(
    real_labels_of_test_dataset,
    predicted_labels
)

sns.heatmap(cm, cmap=sns.dark_palette("#79C"), annot=True)

NameError: name 'y_test' is not defined

Confusion KNN

In [16]:
from sklearn.metrics import confusion_matrix, classification_report

real_labels_of_test_dataset = y_test
predicted_labels = KNN.predict(X_test)
display(confusion_matrix(
    real_labels_of_test_dataset,
    predicted_labels
))

cm = confusion_matrix(
    real_labels_of_test_dataset,
    predicted_labels
)

sns.heatmap(cm, cmap=sns.dark_palette("#79C"), annot=True)

NameError: name 'y_test' is not defined

In [17]:
from sklearn.metrics import confusion_matrix, classification_report

real_labels_of_test_dataset = y_test
predicted_labels = Tree.predict(X_test)
display(confusion_matrix(
    real_labels_of_test_dataset,
    predicted_labels
))

cm = confusion_matrix(
    real_labels_of_test_dataset,
    predicted_labels
)

sns.heatmap(cm, cmap=sns.dark_palette("#79C"), annot=True)

NameError: name 'y_test' is not defined